In [1]:
import numpy as np
from pathlib import Path
import time
import matplotlib.pyplot as plt
start_time = time.time()

import xobjects as xo
import xtrack as xt
import xpart as xp
import xobjects as xo
import xcoll as xc

# import apertls
from matplotlib.colors import LogNorm

import gzip
from typing import Union

import awkward as ak
import pandas as pd

# Line 

In [2]:
line = xt.Line.from_json('../injection_lines/sps_with_aperture_inj_q20_beam_sagitta.json')

tt = line.get_table()
tw = line.twiss()

#links for bump
env = line.env

line['bump.11207'] = 0
line['bump.11407'] = 0
line['bump.11607'] = 0
line['bump.12207'] = 0

line.ref['mdh.11207'].knl[0] += line.vars['bump.11207']
line.ref['mdh.11407'].knl[0] += line.vars['bump.11407']
line.ref['mdh.11607'].knl[0] += line.vars['bump.11607']
line.ref['mdh.12207'].knl[0] += line.vars['bump.12207']

#tidp drift replacement
s_start_tidp = line.get_s_position('tidp.11434..0')
line.remove(tt.rows['tidp.*'].name)
line.insert(env.place('tidp.11434', at=s_start_tidp), s_tol=1e-6)

Loading line from dict:   0%|          | 0/36381 [00:00<?, ?it/s]

Done loading line from dict.           


Slicing line:   0%|          | 0/75278 [00:00<?, ?it/s]

In [ ]:
# opt = line.match(
#     method='6d',
#     vary=[
#         xt.VaryList(['bump.11207', 'bump.11407', 'bump.11607'], step=1e-8, tag='bump',)
#     ],
#     targets = [
#         xt.Target('x', -21.5e-3, at='tidp.11434'),
#         xt.TargetSet(['x', 'px'], value=tw, at='bph.11608..0')
#     ]
#     )
# tw2 = line.twiss()

# BPM data classes

In [ ]:
from typing import Optional, Tuple, Union, Set, List
import re

class BPMBehavior(ak.Record):
    pass

class BPMCollectionBehavior(ak.Array):

    @property 
    def index(self):
        if not hasattr(self, '_index'):
            self._build_index()
        return self._index
    
    @property
    def bpms_hor(self):
        #return self["orbit", "name"]
        return ak.Array([bpm for i, bpm in enumerate(self['name']) if self['plane'][i] == 'H'])
    
    @property
    def bpms_ver(self):
        return ak.Array([bpm for i, bpm in enumerate(self['name']) if self['plane'][i] == 'V'])
    
    def orbit_plane(self, plane: str = "H"):
        if plane == 'H':
            return ak.Array([self['position'][self.index[bpm]] for bpm in self.bpms_hor])
        elif plane == 'V':
            return ak.Array([self['position'][self.index[bpm]] for bpm in self.bpms_ver])
        else:
            ValueError('No valid plane was given !')
    
    def rms_plane(self, plane: str = "H"):
        if plane == 'H':
            return ak.Array([self['rms'][self.index[bpm]] for bpm in self.bpms_hor])
        elif plane == 'V':
            return ak.Array([self['rms'][self.index[bpm]] for bpm in self.bpms_ver])
        else:
            ValueError('No valid plane was given !')

    def orbit_position(self, name: str):
        # Return position for a given BPM name
        return self['position'][self.index[name]]
    
    def restr_orbit_plane(self, bpm_list_restr):
        return ak.Array([self['position'][self.index[bpm]] for bpm in bpm_list_restr])

    def restr_rms_plane(self, bpm_list_restr):
        return ak.Array([self['rms'][self.index[bpm]] for bpm in bpm_list_restr])
    
    def _build_index(self):
        self._index = {}
        for i, bpm in enumerate(self['name']):
            self._index[bpm] = i

class CorrectorsCollectionBehavior(ak.Array):
    @property 
    def index(self):
        if not hasattr(self, '_index'):
            self._build_index()
        return self._index
    
    @property
    def corr_hor(self):
        #return self["orbit", "name"]
        return ak.Array([corr for i, corr in enumerate(self['name']) if self['plane'][i] == 'H'])
    
    @property
    def corr_ver(self):
        return ak.Array([corr for i, corr in enumerate(self['name']) if self['plane'][i] == 'V'])
    
    def kick_plane(self, plane: str = "H"):
        if plane == 'H':
            return ak.Array([self['kick'][self.index[corr]] for corr in self.corr_hor])
        elif plane == 'V':
            return ak.Array([self['kick'][self.index[corr]] for corr in self.corr_ver])
        else:
            ValueError('No valid plane was given !')

    def kick_position(self, name: str):
        # Return position for a given BPM name
        return self['kick'][self.index[name]]

    def _build_index(self):
        self._index = {}
        for i, corr in enumerate(self['name']):
            self._index[corr] = i


ak.behavior["*", "BPMCollection"] = BPMCollectionBehavior
ak.behavior["*", "CorrectorsCollection"] = CorrectorsCollectionBehavior

In [4]:
def load_bpm_file_data(path: Union[str, Path]) -> ak.Array:
    """
    Load a .data or .data.gz file into an Awkward Array with BPMCollection behavior.
    """

    #file opening with different compression
    path = Path(path)
    opener = gzip.open if path.suffix == ".gz" else open
    with opener(path, "rt") as f:
        lines = f.readlines()

    #Sections of file
    header_lines = []
    orbit_lines = []
    corrector_lines = []

    mode = "header"
    for line in lines:
        if line.startswith("# MONITOR"):
            mode = "orbit"
            continue
        elif line.startswith("# CORRECTOR"):
            mode = "corrector"
            continue

        if mode == "header":
            header_lines.append(line)
        elif mode == "orbit":
            if not line.startswith('*'):
                orbit_lines.append(line)
        elif mode == "corrector":
            if not line.startswith('*'):
                corrector_lines.append(line)

    #Metadata
    meta = {}  # <-- parse @ DATE, @ DPP, etc.
    for line in header_lines:
        if not line.startswith('@'):
            continue
        parts = line.strip().split(maxsplit=3)
        if len(parts) < 4:
            continue
        _, key, dtype, value = parts
        if dtype == "%d":
            value = int(value)
        elif dtype == "%f":
            value = float(value)
        elif dtype == "%s":
            value = value.strip('"')
        meta[key] = ak.Array([value])
    
    
    #Orbit
    orbit = {
        "name": [], "plane": [], "beam": [], "position": [],
        "rms": [], "sum": [], 'hw-status': [], 'status' : [], "status_tag": [],
    }
    # Fill orbit[...] from orbit_lines
    for line in orbit_lines:
        parts = line.strip().split(maxsplit=9)
        assert len(parts)==9
        for i, key in enumerate(list(orbit.keys())):
            value = parts[i]
            # Simple type inference
            try:
                if "." in value:
                    orbit[key].append(float(value))
                else:
                    orbit[key].append(int(value))
            except ValueError:
                if key == 'name':
                    orbit[key].append(value.lower())
                else:
                    orbit[key].append(value)
    
    for key in orbit:
        orbit[key]=ak.Array(orbit[key])
    
    #Correctors
    correctors = {
        "name": [], "plane": [], "beam": [], 'strength_name' : [], "kick": [], "rt_kick": [],
    }
    # Fill correctors[...] from corrector_lines
    for line in corrector_lines:
        parts = line.strip().split(maxsplit=6)
        assert len(parts)==6
        for i, key in enumerate(list(correctors.keys())):
            value = parts[i]
            # Simple type inference
            try:
                if "." in value:
                    correctors[key].append(float(value))
                else:
                    correctors[key].append(int(value))
            except ValueError:
                if key=='name':
                    correctors[key].append(value.lower())
                else:
                    correctors[key].append(value)
    
    for key in correctors:
        correctors[key]=ak.Array(correctors[key])

    #Total dictionary data, idk how to make awkward array out of it
    data = {
        "orbit": ak.Array(orbit, with_name='BPMCollection'),
        "correctors": ak.Array(correctors, with_name='CorrectorsCollection'),
        "meta": ak.Array(meta),
    }

    return data


In [5]:
def load_all_bpm_files(directory_coll):
    bpm_files_by_timestamp = {}

    pattern = re.compile(r"ORBIT_SPSRING_(\d{4}-\d{2}-\d{2}_\d{2}-\d{2}-\d{2})_.*\.data\.gz")

    for file in Path(directory_coll).glob("ORBIT_SPSRING_*.data.gz"):
        match = pattern.match(file.name)
        if match:
            timestamp_raw = match.group(1)
            timestamp = timestamp_raw.replace('_', ' ', 1)  # Convert to 'YYYY-MM-DD HH-MM-SS'
            try:
                bpm_files_by_timestamp[timestamp] = load_bpm_file_data(file)
            except (AssertionError, EOFError, OSError, ValueError) as e:
                print(f"Skipping file due to error ({type(e).__name__}): {file}")
                continue

    return bpm_files_by_timestamp

In [ ]:
# import pytz  # Optional, but good for clarity

# timestamps = list(bpm_files_by_timestamp.keys())
# dt_index = pd.to_datetime(timestamps, format='%Y-%m-%d %H-%M-%S')
# dt_index = dt_index.tz_localize('Europe/Paris')